In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import math
import shutil
import random
import torch

In [5]:
torch.__version__

'1.8.1'

In [2]:
def process_hdr(image):
    return np.minimum(np.maximum(np.log10(image * 0.2 * image.mean() + 1), 0), 2) - 1

def find_power(image, target=0.011):
    l, r = 0.0, 1.0
    m = 0.5
    for i in range(32):
        res = np.power(image, m).mean()
        if abs(res - target) < 1e-6:
            return m
        elif res > target:
            l = m
        else:
            r = m
        m = (r + l) / 2
    return m

In [3]:
src_dir = os.path.join('LavalIndoorHDRDatasetPrepared')
dst_dir = os.path.join('LavalIndoorHDRDatasetReady')

catalog = os.listdir(src_dir)
n = len(catalog)
for i, file in enumerate(catalog):
    data = cv2.resize(np.load(os.path.join(src_dir, file))[:3300,], (2048, 1024))
    data /= data.max()

    target = np.random.rand() * (0.015 - 0.011) + 0.011
    data = np.power(data, find_power(data, target)) * 255
    data = process_hdr(data)

    np.save(os.path.join(dst_dir, file[:-3]), data)
    print('{} / {}'.format(i + 1, n), end='\r')

In [17]:
src_dir = os.path.join('PanoIndoorLDRDataset')
dst_dir = os.path.join('PanoIndoorLDRDatasetReady')

catalog = os.listdir(os.path.join(src_dir, 'bedroom'))
n = len(catalog) - 2
delay = 0
for i, file in enumerate(catalog):
    if file.endswith('.mat'):
        delay += 1
        continue

    data = cv2.imread(os.path.join(src_dir, 'bedroom', file, file + '.jpg'))
    data = cv2.resize(data, (2048, 1024))
    data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    data = process_hdr(data)

    np.save(os.path.join(dst_dir, file), data)
    print('{} / {}'.format(i + 1 - delay, n), end='\r')

catalog = os.listdir(os.path.join(src_dir, 'living_room'))
n = len(catalog) - 2
delay = 0
for i, file in enumerate(catalog):
    if file.endswith('.mat'):
        delay += 1
        continue

    data = cv2.imread(os.path.join(src_dir, 'living_room', file, file + '.jpg'))
    data = cv2.resize(data, (2048, 1024))
    data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB)
    data = process_hdr(data)

    np.save(os.path.join(dst_dir, file), data)
    print('{} / {}'.format(i + 1 - delay, n), end='\r')

In [28]:
for file in os.listdir('LavalIndoorHDRDatasetReady'):
    if file in ('test', 'train'):
        continue
    shutil.move(
        os.path.join('LavalIndoorHDRDatasetReady', file),
        os.path.join('LavalIndoorHDRDatasetReady', 'train' if random() < 0.85 else 'test', file[:-4] + file[-3:])
    )
for file in os.listdir('PanoIndoorLDRDatasetReady'):
    if file in ('test', 'train'):
        continue
    shutil.move(
        os.path.join('PanoIndoorLDRDatasetReady', file),
        os.path.join('PanoIndoorLDRDatasetReady', 'train' if random() < 0.85 else 'test', file)
    )

In [43]:
base = np.zeros((1000, 2000, 3))

In [42]:
def __build_mask(self, base_shape):
    base_shape = (*base_shape, 3)
    image = np.zeros(base_shape)

    for i in range(15):
        scale = random.randint(10, 40)

        h, w, _ = base_shape
        h = h * scale // 100
        w = w * scale // 100

        tmp = np.ones((h, w, 3)) * 255

        angle = random.randint(0, 180)

        M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1)
        tmp = cv2.warpAffine(tmp, M, (w, h))

        h_c = random.randint(base_shape[0] // 3, 2 * base_shape[0] // 3)
        w_c = random.randint(base_shape[1] // 3, 2 * base_shape[1] // 3)

        top_pad = base_shape[0] - h_c - h // 2
        bottom_pad = h_c - h // 2 - h % 2

        right_pad = base_shape[1] - w_c - w // 2
        left_pad = w_c - w // 2 - w % 2

        tmp = np.pad(tmp, ((top_pad, bottom_pad), (left_pad, right_pad), (0, 0)))

        image = cv2.add(image, tmp)

    return image

In [43]:
base = __build_mask(None, (1000, 2000))
cv2.imshow('test', base)
cv2.waitKey()

-1

In [48]:
(base == 0).astype(np.int)

array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       ...,

       [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]],

       [[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        ...,
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]])

In [10]:
class KMeans:
    def __init__(self, K=10, Niter=10):
        self.
    
    def fit(self, x):
        N, D = x.shape
        c = x[:K, :].clone()

        x_i = LazyTensor(x.view(N, 1, D))
        c_j = LazyTensor(c.view(1, K, D))

        for i in range(Niter):
            D_ij = ((x_i - c_j) ** 2).sum(-1)
            cl = D_ij.argmin(dim=1).long().view(-1)

            c.zero_()
            c.scatter_add_(0, cl[:, None].repeat(1, D), x)

            Ncl = torch.bincount(cl, minlength=K).type_as(c).view(K, 1)
            c /= Ncl

        return cl, c

In [3]:
class ORBPatcher:
    def __init__(self, *, key_points=1000):
        self.__orb = cv2.ORB_create(key_points)

    def __call__(self, image):
        key_points, _ = self.__orb.detectAndCompute(image, None)

        res = np.zeros((image.shape[0], image.shape[1]), dtype=np.int)
        for point in key_points:
            x, y = map(np.int, point.pt)
            res[y, x] = 1

        res == 1
        return res.flatten()

In [ ]:
class KMeansFiles:
    def __init__(self, orb, *, clusters=5, n_iter=300, partitions=1):
        self.__orb = orb
        self.__clusters = clusters
        self.__n_iter = n_iter
        self.__partitions = partitions
        self.__centers = None

    def fit(self, files):
        partition = (len(files) + self.__partitions - 1) // self.__partitions
        print('preparing partitions\n')
        for num in range(self.__partitions):
            tmp = self.__prepare_partition(files[num * partition: (num + 1) * partition])
            torch.save(tmp, os.path.join('{}.tmp'.format(num)))
            print('partition {} prepared'.format(num))
        print('preparing partitions\n'.format(len(files)))

        print('fitting k-means on {} samples\n'.format(len(files)))
        for iteration in self.__n_iter:
            for num in range(self.__partitions):
                tmp = torch.load(os.path.join('{}.tmp'.format(num))).to(device)
                print('fitting on partition {}'.format(num))
                self.__fit_partition(tmp)
                print('partition fitted!\n')
            print('iteration {} fitted!\n'.format(iteration))
        print('k-means ready!')

    def __prepare_partition(self, files):
        return torch.tensor([
            self.__orb((((np.load(file)) + 1) / 2 * 255).astype(np.uint8))
            for file in files
        ]).to(device)

    def __fit_partition(self, x):
        n, d = x.shape
        if self.__centers is None:
            self.__centers = x[:self.__clusters, :].clone().to(device)

        x_i = x.view(n, 1, d)
        c_j = self.__centers.view(1, self.__clusters, d)

        D_ij = ((x_i - c_j) ** 2).sum(-1)
        clusters = D_ij.argmin(dim=1).long().view(-1)

        self.__centers.zero_()
        self.__centers.scatter_add_(0, clusters[:, None].repeat(1, D), x)
        self.__centers /= torch.bincount(clusters, minlength=k).type_as(self.__centers).view(self.__clusters, 1)

    def predict(self, patches):
        if self.__centers == None:
            print('k-means wasn\'t fitted')
            return

        n, d = x.shape
        x_i = x.view(n, 1, d)
        c_j = self.__centers.view(1, self.__clusters, d)
        D_ij = ((x_i - c_j) ** 2).sum(-1)
        clusters = D_ij.argmin(dim=1).long().view(-1)

        res = torch.zeros((n, self.__clusters)).to(device)
        for num, item in enumerate(clusters):
            res[num, item] = 1
        return res

In [7]:
def __calc_orb_patch(self, image):
    orb = cv2.ORB_create(1000)
    key_points, _ = orb.detectAndCompute(image, None)

    res = np.zeros_like(image, dtype=np.int8)
    for point in key_points:
        x, y = map(np.int, point.pt)
        res[y, x] = np.array([1,1,1], dtype=np.int8)

    return res

In [9]:
orig = cv2.imread(os.path.join('PanoIndoorLDRDataset', 'bedroom', 'pano_aaacisrhqnnvoq', 'pano_aaacisrhqnnvoq.jpg'))
orig = cv2.resize(orig, (2048, 1024))

cv2.imwrite(os.path.join('Presentation Pictures', 'orb_orig.jpg'), orig)

# cv2.imshow('title', res)
# cv2.waitKey()

True

In [11]:
def prepare_data():
    catalogs = [
        os.path.join('LavalIndoorHDRDatasetReady', 'train'),
        os.path.join('PanoIndoorLDRDatasetReady', 'test'),
        os.path.join('LavalIndoorHDRDatasetReady', 'train'),
        os.path.join('PanoIndoorLDRDatasetReady', 'test')
    ]
    files = []
    for catalog in catalogs:
        files.extend([os.path.join(catalog, file) for file in os.listdir(catalog)])
    return files

In [ ]:
use_cuda = torch.cuda.is_available()
dtype = torch.float32 if use_cuda else torch.float64

KMeans(np.array([
    self.__calc_orb_patch((((np.load(file)) + 1) / 2 * 255).astype(np.uint8))
    for file in prepare_data()
]))

In [2]:
a = np.array([
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1],
])
b = np.array([
    [2, 2, 2],
    [2, 2, 2],
    [2, 2, 2],
])

np.concatenate([a.reshape((3,3,1)), b.reshape(3,3,1)], axis=2)

array([[[1, 2],
        [1, 2],
        [1, 2]],

       [[1, 2],
        [1, 2],
        [1, 2]],

       [[1, 2],
        [1, 2],
        [1, 2]]])

In [6]:
catalog = os.path.join('PanoIndoorLDRDatasetReady', 'train')
im_name = os.listdir(catalog)[50]
im_path = os.path.join(catalog, im_name)

image = np.load(im_path)
cv2.imwrite(os.path.join('Pictures', 'pano1.jpg'), image)

True